In [9]:
# !pip install python-docx

In [1]:
import docx
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer#, util
import glob
import os
import ast
import pandas as pd
import re

from tqdm.auto import tqdm
from llama_cpp import Llama

C:\Users\Profit77\.conda\envs\RAGTemplate\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
PATH = './'
# PATH_DATASETS = PATH + 'datasets/train Atom/train data/'
HMI_PATH_DATASETS = PATH + 'datasets/train Atom/train data/HMI/'
SSTS_PATH_DATASETS = PATH + 'datasets/train Atom/train data/SSTS/'

### Загружаем данные о документах, формируем массив по абзацам/параграфам

In [12]:
hmi_files = glob.glob(HMI_PATH_DATASETS + "/*.docx")
ssts_files = glob.glob(SSTS_PATH_DATASETS + "/*.docx")

# Перечень номеров файлов которые существуют
ssts_filenumbers = list(map(lambda x: os.path.basename(x).split('-')[-1].split('.')[0], ssts_files))
hmi_filenumbers = list(map(lambda x: os.path.basename(x).split('-')[-1].split('.')[0], hmi_files))

len(hmi_files), len(ssts_files)

(12, 11)

In [13]:
%%time
# Загружаем LLM модель
llm_path = "D:/_Projects/Hack_AI_Atom/ggml-model-Q4_K_M.gguf"
llm_model = Llama(model_path=llm_path)
llm_model

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from D:/_Projects/Hack_AI_Atom/ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = a8977699a3d0820e80129fb3c93c20fbd9972c41
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:  

CPU times: total: 922 ms
Wall time: 900 ms


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | RISCV_VECT = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'general.name': 'a8977699a3d0820e80129fb3c93c20fbd9972c41', 'general.architecture': 'llama', 'llama.block_count': '32', 'llama.context_length': '8192', 'tokenizer.ggml.eos_token_id': '128009', 'general.file_type': '15', 'llama.attention.head_count_kv': '8', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.head_count': '32', 'llama.rope.freq_base': '500000.000000', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.vocab_size': '128256', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.model': 'gpt2', 'tokenizer.ggml.pre': 'llama-bpe', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '128000', 'tokenizer.chat_template'

In [23]:
%%time
# Загружаем модель для формиарования эмбеддингов
embedding_model = SentenceTransformer('intfloat/multilingual-e5-large')

CPU times: total: 1.92 s
Wall time: 3.93 s


In [78]:
# Получаем текст из ворда разбитый на параграфы
def get_texts(docx):
    doc_paragraphs = []
    all_paras = docx.paragraphs
    for para in all_paras:
        parag_text = para.text.strip()
        if parag_text != "":
            doc_paragraphs.append(parag_text)
    return doc_paragraphs

In [228]:
## Формируем массив выделенных разделов
##  parag_text_HMI - массив параграфов всего текста ТЗ Закзчика HMI
##  doc_HMI_parts - массив выделенных разделов ТЗ Закзчика HMI
##  parag_text_SSTS - массив параграфов всего текста предложений Исполнителя SSTS
##  doc_SSTS_parts - массив выделенных разделов предложений Исполнителя SSTS

def get_similary_by_doc(model, parag_text, chapters, threshold=0.9):
    # Кодируем документ в эмбеддинг
    embeddings = model.encode(parag_text, convert_to_tensor=True)
    embeddings_chapters = model.encode(chapters, convert_to_tensor=True)
    similarity_chapters = cosine_similarity(embeddings.detach().cpu(), embeddings_chapters.detach().cpu())
    # Получаем отсортированные списки найденных разделов
    find_index_similarity = np.argwhere(similarity_chapters >= threshold)

    # Формируем массив разделов документов с требованиями
    doc_parts = []
    for i in range(len(find_index_similarity)):
        chapt_id = find_index_similarity[i][1]
        start_doc_id = find_index_similarity[i][0]
        if i == len(find_index_similarity) - 1:
            end_doc_id = len(parag_text)
        else:
            end_doc_id = find_index_similarity[i+1][0]
        doc_parts.append({chapters[chapt_id]: parag_text[start_doc_id:end_doc_id]})
    return doc_parts

    
chapters_SSTS = ['Functional Description', 'Enabling conditions', 'Trigger conditions', 'Execution output', 'Exit conditions', 'Notes', 'Basic event flow', 'Optional event flow',]


chapters_HMI = ['Use Case', 'Description', 'Goal', 'Context', 'Scope', 'Actors', 'Triggers', 'Preconditions', 'Main Scenario', 'Postconditions', 'Requirements', 'Alternative Scenario', 'Components', 'Function logic',]
chapters_HMI_weights = {'Use Case': 0, 'Description': 0, 'Goal': 0.5, 'Context': 0, 'Scope': 0, 'Actors': 0, 'Triggers': 0, 'Preconditions': 0, 'Main Scenario': 1.8, 'Postconditions': 0, 'Requirements': 1.0, 'Alternative Scenario': 0.5, 'Components': 0, 'Function logic': 0,} 

# Те разделы которые дают значимость при сопоставлении требований
important_chapters_HMI = [k for k in chapters_HMI_weights if chapters_HMI_weights[k] > 0]
# len(important_chapters_HMI)

# preproc_doc = PreProcessingDoc(embedding_model)
# doc_HMI_parts = get_similary_by_doc(model=model, parag_text=parag_text_HMI, chapters=chapters_HMI, threshold=0.9)
# doc_SSTS_parts = get_similary_by_doc(model=model, parag_text=parag_text_SSTS, chapters=chapters_SSTS, threshold=0.9)

#  parag_text_HMI - массив параграфов всего текста ТЗ Закзчика HMI
#  doc_HMI_parts - массив выделенных разделов ТЗ Закзчика HMI
#  parag_text_SSTS - массив параграфов всего текста предложений Исполнителя SSTS
#  doc_SSTS_parts - массив выделенных разделов предложений Исполнителя SSTS


In [309]:
# LLM подключаем
SYSTEM_PROMPT = """
You are a language model tasked with evaluating how closely a technical specifications (SSTS) aligns with an requirements document (UC). The input format is: 

UC: {scenario requirements content} 

SSTS: {technical specifications for implementation}

Definitions:

    UC represents the requirements document, detailing the scenario requirements for system interface functionality, including preconditions, main scenarios, postconditions, and alternative scenarios.
    SSTS represents a technical document outlining functionality in a structured textual description.
    
Your task is to compare SSTS against the requirements in UC and assess the compliance based on the following criteria. Provide your response *only* in the specified dictionary format, with no introductory or closing text, using these keys:

    Differences: Highlight the key mismatches or omissions in SSTS relative to UC. Focus on:
        Missing or divergent output devices.
        Deviations in user interaction elements, such as control methods.
        Absence of specific functional details such as status display or audio handling requirements.

    Description: Provide a detailed description of missing or divergent requirements to clarify gaps in SSTS. Include key requirements from UC to emphasize the context of the noted differences.

    Compliance Level: Indicate the compliance level of SSTS with UC using two capital letters from this scale:
        FC - Fully Compliant (FC) - The document fully matches all functional requirements in UC with no missing elements.
        LC - Largely Compliant (LC) - The document meets most requirements in UC, with minor deviations noted.
        PC - Partially Compliant (PC) - Some requirements are met, but notable gaps exist that require revisions for alignment.
        NC - Non-compliant (NC) - Significant differences exist, and the document does not meet the specified requirements. A complete revision is needed.
        NA - Not Applicable (NA) - The requirements and document are not comparable due to differences in purpose or scope. Explain the reason.
    
    Compliance Rate: Indicate the compliance rate of SSTS with UC compliance percentage from 0 to 100.

Return your response *only* in the following dictionary format, with no additional text:

```{
    "Differences": "Summary of key differences here",
    "Description": "UC - Key requirements from the interface scenario, SSTS - Description of corresponding content in the technical document",
    "Compliance Level": "FC, LC, PC, NC, or NA"},
    "Compliance Rate": 55
}```

Example format:

```{
    "Differences": "Summary of key differences between SSTS and UC",
    "Description": "UC - Key requirements missing from the technical document, SSTS - Relevant technical document details",
    "Compliance Level": "FC",
    "Compliance Rate": 95
}```
"""



# model_path = "llama-3.2-3b-instruct-q8_0.gguf" # путь к модели
model_path = "ggml-model-Q4_K_M.gguf"

model = Llama(
        model_path=model_path,
        n_ctx=8000,
        n_gpu_layers=-1,
        n_parts=1,
        verbose=False,
    )
#  функция для инференса
def interact(
    user_message,
    top_k=30,
    top_p=0.9,
    temperature=0.1, # регулирует на сколько модель лаконично отвечает
    repeat_penalty=1.1
    
):

    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages.append({"role": "user", "content": user_message})
    result = []
    for part in model.create_chat_completion(
            messages,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repeat_penalty=repeat_penalty,
            stream=True,
        ):
            delta = part["choices"][0]["delta"]
            if "content" in delta:
                result.append(delta["content"])
                
    return ''.join(result).strip()
# interact(model_path)

In [310]:
def compare_chapter_by_specific(chapt_text:str, full_text_SSTS:str):
    while True:  #Цикл до успешного выполнения заданного выхода модели
        res = interact(f'text1: {chapt_text}\n\n\ntext2: {full_text_SSTS}')
        try:
            res = re.sub(r'\s+', ' ', res).replace('```','')
            res = ast.literal_eval(res)
            break  # Если всё прошло успешно, выходим из цикла
        except Exception as e:
            print()
            print(f"Не получилось сразу обработать, пробуем еещ раз: {e}")
            print(res)
            print()
    return res

In [ ]:
from sklearn.metrics import mean_squared_error
def calc_score(gt, sub):
  sub = sub.drop_duplicates(subset='Number', keep='last')

  mapping = {'FC': 1, 'LC': 2, 'PC': 3, 'NC': 4, 'NA': 5}
  gt['категории_числа'] = gt['Complience Level'].map(mapping)
  sub['категории_числа'] = sub['Complience Level'].map(mapping)


  merge_df = pd.merge(gt, sub, on='Number', how='left')
  merge_df['категории_числа_y'] = merge_df['категории_числа_y'].fillna(mapping['NA'])
  mse = mean_squared_error(merge_df['категории_числа_x'], merge_df['категории_числа_y'])

  score = max(0, 1.5-mse)/1.5
  print(f'mse: {mse}')
  print(f'score: {score}')
    
  return score


In [314]:
%%time
# Общий цикл по обработке документов
# compliance_levels = {}
result_submit = []
all_doc_df = pd.DataFrame()

# Основной цикл по прохождению всех документов ТЗ HMI
for hmi_filename in tqdm(hmi_files):
    submit_docs_dict = {}
    print(hmi_filename)
    
    number_hmi = os.path.basename(hmi_filename).split('-')[-1].split('.')[0]
    
    
    # Читаем *.doc документы
    doc_HMI = docx.Document(hmi_filename)
    parag_text_HMI = get_texts(doc_HMI)
    text_number, title = re.findall(r"(\[.*?\])\s+(.*?)$", parag_text_HMI[0])[0]
    # Определяем заголовок из ТЗ
    title = title.strip()
    # compliance_levels[number_hmi] = {'Number':number_hmi, 'Name':title}
    submit_docs_dict['Number'] = number_hmi
    submit_docs_dict['Name'] = title
    
    if number_hmi not in ssts_filenumbers:
        submit_docs_dict['Description'] = '-'
        submit_docs_dict['Differences'] = "ssts hasn't info about this"
        # submit_docs_dict['Compliance Level'] = 'NA'
        submit_docs_dict['average_compliance_rate'] = 0
        result_submit.append(submit_docs_dict)
        continue
   
    # Считываем SSTS файл 
    ssts_filename = ssts_files[ssts_filenumbers.index(number_hmi)]
    # Читаем *.doc документы
    doc_SSTS = docx.Document(ssts_filename)
    parag_text_SSTS = get_texts(doc_SSTS)
    
    # Определяем разделы документов
    doc_HMI_parts = get_similary_by_doc(model=embedding_model, parag_text=parag_text_HMI, chapters=chapters_HMI, threshold=0.9)
    doc_SSTS_parts = get_similary_by_doc(model=embedding_model, parag_text=parag_text_SSTS, chapters=chapters_SSTS, threshold=0.9)
    
    #  parag_text_HMI - массив параграфов всего текста ТЗ Закзчика HMI
    #  doc_HMI_parts - массив выделенных разделов ТЗ Закзчика HMI
    #  parag_text_SSTS - массив параграфов всего текста предложений Исполнителя SSTS
    #  doc_SSTS_parts - массив выделенных разделов предложений Исполнителя SSTS

    # summarize_response = summarize_text(text=full_text, llm_model=llm_model)
    # compliance_levels['Description'] = summarize_response['choices'][0]['text'].strip()
    # compliance_levels['Description'] = # Вызов суммаризатора текста
    # compliance_levels['Differences'] = # Вызов определения разнцы текста
    # compliance_levels['Compliance Level'] = # Вызов определения уровня оценки


    # Формирвем предсказание по всему документу
    full_text_SSTS = '\n '.join(parag_text_SSTS)
    full_text_HMI = '\n '.join(parag_text_HMI)
    
    result_by_chapt = {}
    
    res_all_doc = compare_chapter_by_specific(full_text_HMI, full_text_SSTS)
    res_all_doc['Weight'] = 2
    result_by_chapt['All doc'] = res_all_doc
    
    # Формирвем предсказание по каждому блоку
    for i in tqdm(doc_HMI_parts):
        chapt_key = list(i.keys())[0]
        chapt_text = '\n '.join(list(i.values())[0])
        # Если раздел относится к важным, тогда анализируем его отдельно
        if chapt_key in important_chapters_HMI:
            print(chapt_key)
            print(chapt_text)
            res = compare_chapter_by_specific(chapt_text, full_text_SSTS)
            # compare_part_text_HMI = '\n '.join(list(doc_HMI_parts[0].values())[0])
            res['Weight'] = chapters_HMI_weights[chapt_key]
            result_by_chapt[chapt_key] = res
        # break    
    # Формируем итоговый Датафрейм по документу
    doc_df = pd.DataFrame.from_dict(result_by_chapt, orient='index')
    # doc_df = doc_df.rename(columns={'weight':'Weight'})
    doc_df['Number'] = number_hmi
    doc_df['Name'] = title
    all_doc_df = pd.concat([all_doc_df, doc_df])

    # submit_docs_dict['Number'] = number_hmi
    # submit_docs_dict['Name'] = title
    submit_docs_dict['average_compliance_rate'] = np.average(doc_df['Compliance Rate'].values, weights=doc_df['Weight'].values)
    submit_docs_dict['Differences'] = doc_df.loc['All doc']['Differences']
    submit_docs_dict['Description'] = doc_df.loc['All doc']['Description']
    result_submit.append(submit_docs_dict)
    # break
all_doc_df = all_doc_df.dropna()    

  0%|          | 0/12 [00:00<?, ?it/s]

./datasets/train Atom/train data/HMI\UC-11467.docx

Не получилось сразу обработать, пробуем еещ раз: invalid syntax (<unknown>, line 1)
Here is the evaluation: {"Differences": "No specific differences found in this section.", "Description": "", "Compliance Level": "FC", "Compliance Rate": 100}  The provided text1 and text2 describe the technical specifications for revoking access to a vehicle and remotely controlling digital keys (DK) respectively. Text1 provides clear requirements for revoking access from drivers/passengers, with specific steps outlined in the main scenario and alternative scenarios. The rights given settings are saved in the user profile and all accesses previously generated by this profile are also deleted if applicable. Text2 provides technical specifications for remotely controlling DK deletion through the ATOM app. It includes enabling conditions, trigger conditions, execution output, basic event flow, and optional event flow. There are no specific differences fo

  0%|          | 0/5 [00:00<?, ?it/s]

Goal
Goal:
 Enable the Owner to quickly revoke access from other drivers/profiles to the vehicle through the ATOM application.

Не получилось сразу обработать, пробуем еещ раз: unterminated string literal (detected at line 1) (<unknown>, line 1)
{"Differences": "Summary of key differences between SSTS and UC", "Description": "UC - Key requirements from the interface scenario, SSTS - Description of corresponding content in the technical document", "Compliance Level": "LC", "Compliance Rate": 80} Differences: - Absence of specific functional details such as status display or audio handling requirements. Description: The UC describes a specific goal and conditions for revoking access to the vehicle through the ATOM application. In contrast, SSTS focuses on the technical flow of events related to remote DK deletion, without explicitly mentioning the owner's ability to quickly revoke access or providing detailed functional requirements such as status display or audio handling. Compliance Le

  0%|          | 0/3 [00:00<?, ?it/s]

...

Preconditions
Preconditions:
 ATOM is on
 in_5. Smartphone is paired with the car
Main Scenario
Main Scenario:
 User selects the "Multimedia" via in_5 or in_2/
 out_2/out_5 displays the available audio sources:
 Internet Radio
 FM Radio
 USB
 Internal SD card
 Bluetooth
Postconditions
Postconditions:
 out_2/out_5 shows available media sources
Requirements
Requirements:
 The action is displayed on the touchpoint from which this action is performed
./datasets/train Atom/train data/HMI\UC-8692.docx


  0%|          | 0/5 [00:00<?, ?it/s]

Preconditions
Preconditions:
 The vehicle is in the Drive State (PRND)
 The vehicle is capable of communicating with emergency services (ERA-Glonass functional).
 The vehicle is involved in or witnesses an emergency, such as a collision or a medical issue, necessitating communication with emergency services.
Triggers
Trigger:
 The vehicle sensors detect a collision and activate the emergency communication feature.
 The driver manually presses in_19. SOS button (roof console)activate the emergency communication feature.
Main Scenario
Main Scenario:
 The ERA-Glonass initiates emergency service communication.
 During the emergency call, the safety audio system automatically mutes or lowers the volume of other audio sources within the vehicle, such as the radio, navigation prompts, or entertainment systems.
 The public safety answering point (PSAP) receives critical information about the situation, location, and the nature of the emergency from the In-vehicle system and occupants' answers.

  0%|          | 0/5 [00:00<?, ?it/s]

Preconditions

Postconditions
Preconditions
 Entertainment system is ready. in_2
 The driver's smartphone is connected to the vehicle's system via Bluetooth (in_32)[I-6323] Pairing a Mobile Phone.
Main Scenario
Main Scenario:
 An incoming call is received on the driver's phone (in_5).
 The caller's information (name from the contact list or phone number) is displayed on the SWP Android (out_2).
 The interior ambient light (out_12) changes color to signal the incoming call.
 An incoming call sound is played through the vehicle's speakers (out_24).
 The driver can accept or reject the call using the front buttons on the steering wheel (in_10) or via the voice assistant command (in_24).
Postconditions
Postconditions:
 If the call is accepted, the interior ambient light changes color, the call status is displayed on the HUD and SWP Android, and the vehicle's microphone (in_24) and speakers out_24 are used for the call.
 If the call is rejected, the interior ambient light returns to its ori

In [602]:
# all_doc_df[(all_doc_df['Number'] == number)&(all_doc_df.index.isin(['All doc', 'Goal', 'Main Scenario', 'Requirements', 'Alternative Scenario']))]
tmp_all_doc_df = all_doc_df.copy()
tmp_all_doc_df['Type'] = tmp_all_doc_df.index
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'All doc', 'Weight'] = 1
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Goal', 'Weight'] = 0.5
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Preconditions', 'Weight'] = 0
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Main Scenario', 'Weight'] = 1.8 # 2
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Alternative Scenario', 'Weight'] = 0.5
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Postconditions', 'Weight'] = 0
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Requirements', 'Weight'] = 1 #2
tmp_all_doc_df.loc[tmp_all_doc_df['Type'] == 'Triggers', 'Weight'] = 0
# tmp_all_doc_df

union_docs_dict = {}
for number in tmp_all_doc_df['Number'].unique():
    union_docs_dict[number] = {}
    union_docs_dict[number]['Number'] = number
    # union_docs_dict[number]['average_compliance_rate'] = tmp_all_doc_df[tmp_all_doc_df['Number'] == number]['Compliance Rate'].mean()
    select_df = tmp_all_doc_df[(tmp_all_doc_df['Number'] == number)&(tmp_all_doc_df.index.isin(['All doc', 'Goal', 'Main Scenario', 'Requirements', 'Alternative Scenario']))]
    # union_docs_dict[number]['average_compliance_rate'] = select_df['Compliance Rate'].mean()
    union_docs_dict[number]['average_compliance_rate'] = np.average(select_df['Compliance Rate'].values, weights=select_df['Weight'].values)
    # print(number)
    # chapters_HMI_weights = {'Use Case': 0, 'Description': 0, 'Goal': 3.1, 'Context': 1.7, 'Scope': 0, 'Actors': 0, 'Triggers': 1.2, 'Preconditions': 1.3, 'Main Scenario': 3.1, 'Postconditions': 1.3, 'Requirements': 2.0, 'Alternative Scenario': 2.9, 'Components': 0, 'Function logic': 0,} 
    # break

union_docs_df = pd.DataFrame().from_dict(union_docs_dict, orient='index')
union_docs_df['Number'] = union_docs_df['Number'].astype(int)
union_docs_df['Complience Level'] = np.where(union_docs_df['average_compliance_rate'] > 80, 'FC', 
        np.where(union_docs_df['average_compliance_rate'] > 70, 'LC', 
            np.where(union_docs_df['average_compliance_rate'] > 50, 'PC', 
                np.where(union_docs_df['average_compliance_rate'] > 30, 'NC', 'NA')
            )
        )
    )
# union_docs_df.info()
calc_score(gt, union_docs_df)

# all_doc_df = all_doc_df.dropna() 

mse: 0.6666666666666666
score: 0.5555555555555556


0.5555555555555556